In [107]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from helper_functions import data_load, data_split, rf_model, google_export
import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [108]:
year = 2024
week = 6
day = 0 # 0-Sun, 1-Mon, 4-Thu

In [109]:
allSeasons, currSeason = data_load(year, week)

In [110]:
features = allSeasons.drop(['Under', 'Push', 'gameday', 'location', 'stadium_id', 'game_type', 'game_id', 'surface', 'home_score', 'away_score', 'result', 'total', 'overtime', 'old_game_id', 'gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_qb_id', 'home_qb_id', 'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee', 'stadium', 'wind', 'temp'], axis=1).columns

KeyError: "['location', 'stadium_id', 'game_type'] not found in axis"

# Accuracy Testing

In [ ]:
# Model building
df_acc = allSeasons.dropna()
df_acc.reset_index(drop=True, inplace=True)
y = df_acc.Under
X = df_acc[features]

precis_array = []
acc_array = []
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
for i in range(1, 26):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=i, stratify=y)
    for j in range(1,11):
        rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=j)
        rf.fit(X_train, y_train)
        preds = rf.predict(X_test)
        acc = accuracy_score(y_test, preds)
        precis = precision_score(y_test, preds)
        precis_array.append(precis)
        acc_array.append(acc)
# combined = pd.DataFrame(dict(actual=y_test, prediction=preds))
# pd.crosstab(index=combined['actual'], columns=combined['prediction'])

In [ ]:
def find_mean(array):
    if len(array) == 0:
        return 0  # To handle empty arrays
    return sum(array) / len(array)
find_mean(precis_array)

0.596502913317647

In [ ]:
find_mean(acc_array)

0.5502222222222222

# Weekly Plays

In [ ]:
X_train, y_train, X_test, y_test = data_split(allSeasons, features, year, week, day)

prediction_df = rf_model(X_train, y_train, X_test)

# Predicted Plays log
nextPlays = pd.merge(right=prediction_df, left=currSeason, right_index=True, left_index=True, how='left')
nextPlays = nextPlays[nextPlays.Prediction == 1]
nextPlays = nextPlays[['game_id', 'season_x', 'week_x', 'home_team', 'away_team', 'gametime_x', 'weekday_x', 'total_line_x', 'under_odds_x']]
nextPlays.columns = ['Game ID', 'Season', 'Week', 'Home', 'Away', 'Start Time', 'Day', 'Total Line', 'Under Odds']
nextPlays

,Game ID,Season,Week,Home,Away,Start Time,Day,Total Line,Under Odds
6785,2024_06_JAX_CHI,2024,6,CHI,JAX,09:30,Sunday,44.5,-110.0
6787,2024_06_ARI_GB,2024,6,GB,ARI,13:00,Sunday,47.0,-108.0
6789,2024_06_TB_NO,2024,6,NO,TB,13:00,Sunday,42.0,-110.0
6791,2024_06_IND_TEN,2024,6,TEN,IND,13:00,Sunday,43.0,-110.0
6792,2024_06_LAC_DEN,2024,6,DEN,LAC,16:05,Sunday,35.5,-108.0
6793,2024_06_PIT_LV,2024,6,LV,PIT,16:05,Sunday,36.5,-108.0
6794,2024_06_ATL_CAR,2024,6,CAR,ATL,16:25,Sunday,47.0,-112.0
6795,2024_06_DET_DAL,2024,6,DAL,DET,16:25,Sunday,52.5,-110.0
6796,2024_06_CIN_NYG,2024,6,NYG,CIN,20:20,Sunday,47.0,-112.0


In [ ]:
X_test

,season,week,weekday,gametime,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,month,game_type_DIV,game_type_REG,game_type_SB,game_type_WC,location_Neutral,stadium_id_BAL00,stadium_id_BOS00,stadium_id_BUF00,stadium_id_CAR00,stadium_id_CHI98,stadium_id_CIN00,stadium_id_CLE00,stadium_id_DAL00,stadium_id_DEN00,stadium_id_DET00,stadium_id_FRA00,stadium_id_GER00,stadium_id_GNB00,stadium_id_HOU00,stadium_id_IND00,stadium_id_JAX00,stadium_id_KAN00,stadium_id_LAX01,stadium_id_LON00,stadium_id_LON02,stadium_id_MEX00,stadium_id_MIA00,stadium_id_MIN01,stadium_id_NAS00,stadium_id_NOR00,stadium_id_NYC01,stadium_id_PHI00,stadium_id_PHO00,stadium_id_PIT00,stadium_id_SAO00,stadium_id_SEA00,stadium_id_SFO01,stadium_id_TAM00,stadium_id_VEG00,stadium_id_WAS00,home_team_ATL,home_team_BAL,home_team_BUF,home_team_CAR,home_team_CHI,home_team_CIN,home_team_CLE,home_team_DAL,home_team_DEN,home_team_DET,home_team_GB,home_team_HOU,home_team_IND,home_team_JAX,home_team_KC,home_team_LA,home_team_LAC,home_team_LV,home_team_MIA,home_team_MIN,home_team_NE,home_team_NO,home_team_NYG,home_team_NYJ,home_team_PHI,home_team_PIT,home_team_SEA,home_team_SF,home_team_TB,home_team_TEN,home_team_WAS,away_team_ATL,away_team_BAL,away_team_BUF,away_team_CAR,away_team_CHI,away_team_CIN,away_team_CLE,away_team_DAL,away_team_DEN,away_team_DET,away_team_GB,away_team_HOU,away_team_IND,away_team_JAX,away_team_KC,away_team_LA,away_team_LAC,away_team_LV,away_team_MIA,away_team_MIN,away_team_NE,away_team_NO,away_team_NYG,away_team_NYJ,away_team_PHI,away_team_PIT,away_team_SEA,away_team_SF,away_team_TB,away_team_TEN,away_team_WAS,Prediction
6785,2024,6,Sunday,34200,7,7,-102.0,-118.0,1.0,-112.0,-108.0,44.5,-110.0,-110.0,0,0,10,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6786,2024,6,Sunday,46800,7,7,250.0,-310.0,7.0,-115.0,-105.0,51.5,-110.0,-110.0,0,0,10,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6787,2024,6,Sunday,46800,7,7,195.0,-238.0,5.5,-112.0,-108.0,47.0,-108.0,-112.0,0,0,10,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6788,2024,6,Sunday,46800,7,7,-285.0,230.0,-6.5,-112.0,-108.0,37.5,-110.0,-110.0,0,0,10,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6789,2024,6,Sunday,46800,10,6,-175.0,145.0,-3.5,-105.0,-115.0,42.0,-110.0,-110.0,1,1,10,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
6790,2024,6,Sunday,46800,7,14,360.0,-470.0,9.5,-110.0,-110.0,42.5,-108.0,-112.0,0,0,10,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6791,2024,6,Sunday,46800,7,13,120.0,-142.0,3.0,-118.0,-102.0,43.0,-110.0,-110.0,1,0,10,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6792,2024,6,Sunday,57900,14,7,-148.0,124.0,-3.0,-105.0,-115.0,35.5,-108.0,-112.0,1,0,10,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0